# Regions use in existing evaluation scheme
Here we will showcase the example where there is an existing evaluation tools that calculates IOU and RMSE for binary and prediction evaluation. The idea is that we do not want to replace the existing methods but supplement its evaluation by using the regions approach.

In [ ]:
import importlib
from functools import partial
from utils import *

root_dir = add_root_to_path(2)

from sep.inspect.utils import *
import sep.inspect.regions as regions_inspect
import sep.assessors.regions as regions
importlib.reload(regions)
import sep._commons.visuals as visuals
import imageio

## Prepare input data

In [ ]:
import sep.loaders.images
data_root = pick_path([root_dir / "tests" / "input" / "basics"])
example_data_dir = pick_path([root_dir / "examples" / "using_regions_data"])
dataset = data_root / "humans"
data_loader = sep.loaders.images.ImagesLoader.from_tree(str(dataset))

show(data_loader[0]['image'], data_loader[1]['image'], data_loader[2]['image'])

## Load segmentations

In [ ]:
image = data_loader[0]['image']
annotation = data_loader[0]['annotation']
seg_1_low_res = imageio.imread(example_data_dir / "human_1_seg_low_res.png")
seg_1_no_hands = imageio.imread(example_data_dir / "human_1_seg_no_hand.png")
seg_1_no_robust = imageio.imread(example_data_dir / "human_1_seg_non_robust.png")
show(annotation, seg_1_low_res, seg_1_no_hands, seg_1_no_robust)

## Show regions for the datasets

In [ ]:
regioners_standard = regions.set_standard
regioners_dict = {str(regioner): regioner.extract_region for regioner in regioners_standard}

In [ ]:
importlib.reload(sep.inspect.regions)
from sep.inspect.regions import show_regions_and_fractions
def calculate_region_proportionals(data_loader, region):
    res = []
    for data_sample in data_loader:
        res.append(region.area_fractions(data_sample['annotation']))
    return res

def show_regions_proportions(data_loader, region):
    printmd(f"*{str(region)}*")
    metrics = calculate_region_proportionals(data_loader, region)
    plt.plot(list(range(len(metrics))), metrics)
    plt.show()

### Inspect fraction of the regions

In [ ]:
show_regions_proportions(data_loader, regioners_standard[0])
show_regions_proportions(data_loader, regioners_standard[1])
show_regions_proportions(data_loader, regioners_standard[2])
show_regions_proportions(data_loader, regioners_standard[3])
show_regions_proportions(data_loader, regioners_standard[4])

### Inspect region masks

In [ ]:
show_regions_and_fractions(regioners_standard,data_loader[0]['annotation'])

In [ ]:
show_regions_and_fractions(regioners_standard,data_loader[1]['annotation'])

In [ ]:
show_regions_and_fractions(regioners_standard,data_loader[2]['annotation'])

### Inspect using napari viewer
Here you can take a look at the regions in napari.

In [ ]:
# Using listing.
#!python -m sep.examine regions_for_dataset <data_root_path> <listing_file_path>

# Based on the already used loader:
import sep.examine
sep.examine.inspect_regions(data_loader)

## Use region to calculate metrics

In [ ]:
from sep.assessors.metricer import Metricer
from sep.assessors.metrics import IouMetric, RMSEMetric
from sep.assessors.regions import EdgesRegion

regioners_standard = regions.set_standard

metricer = Metricer()
metricer.metrics.append(IouMetric())
metricer.metrics.append(RMSEMetric())
for region in regioners_standard[1:]:
    metricer.regions.append(region)

print("On entire image and near the edges:")
data_point_eval = metricer.calculate_metrics(seg_1_low_res > 128, annotation)

In [ ]:
from sep.assessors.regions import RegionExpr
reg = RegionExpr('~', EdgesRegion(0.02), name="Mask robust2")  # #regioners_standard[3]
print(str(reg))
seg_region = reg.regionize(ground_truth=annotation, mask=seg_1_low_res > 128)
gt_region = reg.regionize(ground_truth=annotation, mask=annotation)
region_mask = reg.extract_region(annotation)
result = annotation.astype(bool) & region_mask

print(IouMetric().calculate(seg_1_low_res > 128, annotation))
print(IouMetric().calculate(seg_region, gt_region))
show(image, seg_region, gt_region, region_mask, result)

In [ ]:
show(image, annotation, seg_1_low_res)

In [ ]:
regioners_standard[1].regionize(annotation, seg_1_low_res)

In [ ]:
seg_1_low_res.dtype, annotation.dtype

In [ ]:
np.unique(seg_1_low_res), np.unique(annotation)

In [ ]:
data_point_eval